In [1]:
%load_ext autoreload
%autoreload 2

## Nora et al. 2017

GEO: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE98671

Paper: https://doi.org/10.1016/j.cell.2017.05.004

### Preprocessing

Commands in `COMMANDS` file

### Notes

* AID: Auxin inducible degron

In [2]:
import os
import resgen as rg

rgc = rg.connect(
  os.getenv('RESGEN_USER'),
  os.getenv('RESGEN_PASSWORD')
)

In [3]:
project = rgc.find_or_create_project('Nora et al. 2017', group='paper-data')

In [4]:
project.sync_dataset('ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM2609nnn/GSM2609185/suppl/GSM2609185_CTCF_ChIP-seq_CTCF-AID_untreated_rep1_ENC124_tagDensity.bw',
                    assembly='mm9', datatype='vector', filetype='bigwig')
project.sync_dataset('ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM2609nnn/GSM2609186/suppl/GSM2609186_CTCF_ChIP-seq_CTCF-AID_auxin2days_rep1_ENC125_tagDensity.bw',
                    assembly='mm9', datatype='vector', filetype='bigwig')
project.sync_dataset('ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM2609nnn/GSM2609187/suppl/GSM2609187_CTCF_ChIP-seq_CTCF-AID_washoff_rep1_ENC126_tagDensity.bw',
                    assembly='mm9', datatype='vector', filetype='bigwig')

'BRDX_k8bQKO0IGcTHlyHmA'

In [5]:
project.sync_dataset('GSM2644945_Untreated-R1.20000.mcool', assembly='mm9', datatype='matrix', filetype='cooler')
project.sync_dataset('GSM2644947_Auxin2days-R1.20000.mcool', assembly='mm9', datatype='matrix', filetype='cooler')
project.sync_dataset('GSM2644949_Washoff2days-R1.20000.mcool', assembly='mm9', datatype='matrix', filetype='cooler')

'FH6V19oMS1WZIueLp5Vrrw'

## Figures

In [6]:
chromsizes_mm9 = rgc.find_datasets(
  datatype='chromsizes', assembly='mm9'
)[0]
chrominfo = rgc.get_chrominfo(chromsizes_mm9)

In [7]:
datasets = rgc.find_datasets(project=project, limit=20)
ds_dict = dict([(d.name, d) for d in datasets])
ds_dict

{'GSM2609185_CTCF_ChIP-seq_CTCF-AID_untreated_rep1_ENC124_tagDensity.bw': MQxdUXHb: GSM2609185_CTCF_ChIP-seq_CTCF-AID_untreated_rep1_ENC124_tagDensity.bw,
 'GSM2609186_CTCF_ChIP-seq_CTCF-AID_auxin2days_rep1_ENC125_tagDensity.bw': X-tjLqf7: GSM2609186_CTCF_ChIP-seq_CTCF-AID_auxin2days_rep1_ENC125_tagDensity.bw,
 'GSM2609187_CTCF_ChIP-seq_CTCF-AID_washoff_rep1_ENC126_tagDensity.bw': BRDX_k8b: GSM2609187_CTCF_ChIP-seq_CTCF-AID_washoff_rep1_ENC126_tagDensity.bw,
 'GSM2644945_Untreated-R1.20000.mcool': ZteVkOQQ: GSM2644945_Untreated-R1.20000.mcool,
 'GSM2644947_Auxin2days-R1.20000.mcool': cpevKJDs: GSM2644947_Auxin2days-R1.20000.mcool,
 'GSM2644949_Washoff2days-R1.20000.mcool': FH6V19oM: GSM2644949_Washoff2days-R1.20000.mcool}

In [15]:
import higlass
from higlass.client import View, Track

initialXDomain = [
    chrominfo.to_abs('chr1', 16.6e6),
    chrominfo.to_abs('chr1', 17.8e6)
]

options = {
    "height": 240,
    "colorbarPosition": "hidden",
}

ctcf_options = {
    "valueScaleMax": 20,
    "name": "CTCF ChIP-seq",
}

v1 = View(
    [
        ds_dict['GSM2609185_CTCF_ChIP-seq_CTCF-AID_untreated_rep1_ENC124_tagDensity.bw'].hg_track(
            **ctcf_options,
        ),
        ds_dict['GSM2644945_Untreated-R1.20000.mcool'].hg_track(
        **{**options, "name": "untreated"},
        ),
        chromsizes_mm9.hg_track(position="bottom")
    ],
    initialXDomain=initialXDomain,
    width=4,
)

v2 = View(
    [
        ds_dict['GSM2609186_CTCF_ChIP-seq_CTCF-AID_auxin2days_rep1_ENC125_tagDensity.bw'].hg_track(
            **ctcf_options,
        ),
        ds_dict['GSM2644947_Auxin2days-R1.20000.mcool'].hg_track(
        **{**options, "name": "auxin 2 days"}
        ),
        chromsizes_mm9.hg_track(position="bottom")
    ],
    initialXDomain=initialXDomain,
    width=4,
    x=4
)

v3 = View(
    [
        ds_dict['GSM2609187_CTCF_ChIP-seq_CTCF-AID_washoff_rep1_ENC126_tagDensity.bw'].hg_track(
            **ctcf_options,
        ),
        ds_dict['GSM2644949_Washoff2days-R1.20000.mcool'].hg_track(
        **{**options, "colorbarPosition": "topRight", "name": "auxin 2 days washoff 2 days"}
        ),
        chromsizes_mm9.hg_track(position="bottom")
    ],
    initialXDomain=initialXDomain,
    width=4,
    x=8
)

views = [v1,v2,v3]
(d,s,v) = higlass.display(views,
                          zoom_syncs=[views],
                          location_syncs=[views],
                          value_scale_syncs=[views])
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'LVakZgk-SraPnibyfGdPkQ', 'tracks': {'top': [{'ty…

In [16]:
project.sync_viewconf(v, "Figure 2A")